In [1]:
%run Requirements.ipynb

24/05/23 21:58:44 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
24/05/23 21:58:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 21:58:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/23 21:58:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/23 21:58:46 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
%run secrets.ipynb

In [3]:
df = pd.read_csv('TopGlobalArtists50.csv')
selected_column = df['ArtistName']
selected_column.to_csv('ArtistList.csv', index=False)

Get the Total Reach of Artist From Last FM After Integrating with Spotify

In [11]:
artist_names_df = pd.read_csv("ArtistList.csv")
artist_data_list = []
api_key = Secrets.get('lastFM_APIkey')

for index, row in artist_names_df.iterrows():
    artist_name = row["ArtistName"]
    
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={api_key}&format=json"
    response = requests.get(url)
    artist_info = response.json()
    artist_data = {}
    if "artist" in artist_info:
        artist = artist_info["artist"]
        artist_data["ArtistName"] = artist.get("name")
        stats = artist.get("stats", {})
        artist_data["Listeners"] = stats.get("listeners")
        artist_data["PlayCount"] = stats.get("playcount")
        tags = artist.get("tags", {}).get("tag", [])
        genres = [tag["name"] for tag in tags]
        artist_data["Genres"] = ", ".join(genres)
        artist_data_list.append(artist_data)
    else:
        print(f"No artist information found for {artist_name}.")
artist_df = pd.DataFrame(artist_data_list)
artist_df.to_parquet("Mart/Artists_LastFM", index=False)

Get Track wise Rreach of Artist From Last FM after integrating with Spotify

In [13]:
spotify_tracks_df = pd.read_csv("TopGlobalArtists50.csv")

def get_track_info(artist_name, track_name):
    api_key = "130ad717055f46130e0be133f8497c20"
    url = f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={api_key}&artist={artist_name}&track={track_name}&format=json"
    response = requests.get(url)
    track_info = response.json().get("track", {})
    track_id = track_info.get("mbid")
    duration = track_info.get("duration")
    listeners = track_info.get("listeners")
    play_count = track_info.get("playcount")
    album = track_info.get("album", {}).get("title")
    tags = [tag["name"] for tag in track_info.get("toptags", {}).get("tag", [])]
    
    return {
        "TrackName": track_name,
        "ArtistName": artist_name,
        "TrackID": track_id,
        "Duration": duration,
        "Listeners": listeners,
        "PlayCount": play_count,
        "Album": album,
        "Tags": ", ".join(tags)
    }

track_info_list = []

for index, row in spotify_tracks_df.iterrows():
    artist_name = row["ArtistName"]
    track_name = row["track"]
    track_info = get_track_info(artist_name, track_name)
    track_info_list.append(track_info)

track_info_df = pd.DataFrame(track_info_list)
track_info_df.to_parquet("Mart/Tracks_LastFM", index=False)


Get the information regarding the Albums of the Artist from the Last FM after integraing with Spotify

In [14]:
spotify_albums_df = pd.read_csv("TopGlobalArtists50.csv")

def get_album_info(artist_name, album_name):
    api_key = "130ad717055f46130e0be133f8497c20"
    url = f"http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={api_key}&artist={artist_name}&album={album_name}&format=json"
    response = requests.get(url)
    
    try:
        album_info = response.json().get("album", {})
    except ValueError:
        print(f"Invalid response for artist '{artist_name}' and album '{album_name}': {response.text}")
        return None

    album_id = album_info.get("mbid")
    listeners = album_info.get("listeners")
    play_count = album_info.get("playcount")
    tags = [tag["name"] for tag in album_info.get("toptags", {}).get("tag", [])]
    release_date = album_info.get("wiki", {}).get("published")

    return {
        "AlbumName": album_name,
        "ArtistName": artist_name,
        "AlbumID": album_id,
        "Listeners": listeners,
        "PlayCount": play_count,
        "Tags": ", ".join(tags),
        "ReleaseDate": release_date
    }

album_info_list = []

for index, row in spotify_albums_df.iterrows():
    artist_name = row["ArtistName"]
    album_name = row["album"] 
    album_info = get_album_info(artist_name, album_name)
    if album_info:
        album_info_list.append(album_info)

album_info_df = pd.DataFrame(album_info_list)
album_info_df.to_parquet("Mart/Albums_LastFM", index=False)